NAME: EDUCACIÓN MEN <br>
DESCRIPTION: data import and cleaning administrative data from Ministerio de Educación Nacional (MEN) of Colombia<br>
PROJECT: Childhood in Barranquilla - Econometria Consultores<br>
AUTHOR: Juan José Rincón  <br>

In [1]:
import pandas as pd
import numpy as np
import os
from re import sub
from sklearn import preprocessing
from statistics import mean, stdev

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 4000

In [3]:
os.chdir("/Users/JuanJose/Library/CloudStorage/GoogleDrive-j.rincon@econometria.com/Mi unidad/Libro Niñez DS")

In [4]:
MEN = pd.read_csv(
    "datos educacion media/MEN_ESTADISTICAS_EN_EDUCACION_EN_PREESCOLAR__B_SICA_Y_MEDIA_POR_MUNICIPIO.csv")
MEN.rename(columns = {"CÓDIGO_MUNICIPIO":"COD_MUN"}, inplace = True)

MEN = MEN.loc[(MEN['COD_MUN'] == 11001) | (MEN['COD_MUN'] == 5001)  | (MEN['COD_MUN'] == 76001) |
              (MEN['COD_MUN'] == 8001)  | (MEN['COD_MUN'] == 13001) | (MEN['COD_MUN'] == 44001) | 
              (MEN['COD_MUN'] == 23001) | (MEN['COD_MUN'] == 47001) | (MEN['COD_MUN'] == 20001) | 
              (MEN['COD_MUN'] == 70001) | (MEN['COD_MUN'] == 68001) | (MEN['COD_MUN'] == 54001) | 
              (MEN['COD_MUN'] == 73001) | (MEN['COD_MUN'] == 50001) | (MEN['COD_MUN'] == 66001) | 
              (MEN['COD_MUN'] == 17001) | (MEN['COD_MUN'] == 52001)]

MEN = MEN.assign(
    AÑO = pd.to_numeric(MEN.AÑO.str.replace(',', '')),
    POBLACIÓN_5_16 = pd.to_numeric(MEN.POBLACIÓN_5_16.str.replace(',', '')))

MEN= MEN[['AÑO',"COD_MUN", "MUNICIPIO",'POBLACIÓN_5_16','COBERTURA_NETA','COBERTURA_NETA_TRANSICIÓN',
          'COBERTURA_NETA_PRIMARIA','COBERTURA_NETA_SECUNDARIA','COBERTURA_NETA_MEDIA','COBERTURA_BRUTA',
          'COBERTURA_BRUTA_TRANSICIÓN','COBERTURA_BRUTA_PRIMARIA','COBERTURA_BRUTA_SECUNDARIA',
          'COBERTURA_BRUTA_MEDIA','DESERCIÓN','DESERCIÓN_TRANSICIÓN','DESERCIÓN_PRIMARIA','DESERCIÓN_SECUNDARIA',
          'DESERCIÓN_MEDIA','APROBACIÓN','APROBACIÓN_TRANSICIÓN','APROBACIÓN_PRIMARIA','APROBACIÓN_SECUNDARIA',
          'APROBACIÓN_MEDIA','REPROBACIÓN','REPROBACIÓN_TRANSICIÓN','REPROBACIÓN_PRIMARIA',
          'REPROBACIÓN_SECUNDARIA','REPROBACIÓN_MEDIA','REPITENCIA','REPITENCIA_TRANSICIÓN','REPITENCIA_PRIMARIA',
          'REPITENCIA_SECUNDARIA','REPITENCIA_MEDIA']]


In [5]:
conditions = [(MEN['COD_MUN'] == 11001) | (MEN['COD_MUN'] == 5001) | (MEN['COD_MUN'] == 76001),
              (MEN['COD_MUN'] == 8001),
              (MEN['COD_MUN'] == 13001) | (MEN['COD_MUN'] == 44001) | (MEN['COD_MUN'] == 23001) | 
              (MEN['COD_MUN'] == 47001) | (MEN['COD_MUN'] == 20001) | (MEN['COD_MUN'] == 70001),
              (MEN['COD_MUN'] == 68001) | (MEN['COD_MUN'] == 54001) | (MEN['COD_MUN'] == 73001) | 
              (MEN['COD_MUN'] == 50001) | (MEN['COD_MUN'] == 66001) | (MEN['COD_MUN'] == 17001) | 
              (MEN['COD_MUN'] == 52001)
             ]

choices = ['PRINCIPALES', 'BARRANQUILLA', 'CARIBE','OTRAS']
MEN["GRUPO"]= np.select(conditions, choices)


In [6]:
MEN_Poblacion_5_16=MEN[['COD_MUN','AÑO','POBLACIÓN_5_16']]
MEN_Poblacion_5_16.to_csv("datos educacion media/MEN_Poblacion_5_16.csv")


In [7]:
MEN_ESTADISTICAS=['COBERTURA_NETA','COBERTURA_NETA_TRANSICIÓN','COBERTURA_NETA_PRIMARIA',
                  'COBERTURA_NETA_SECUNDARIA','COBERTURA_NETA_MEDIA','COBERTURA_BRUTA',
                  'COBERTURA_BRUTA_TRANSICIÓN','COBERTURA_BRUTA_PRIMARIA','COBERTURA_BRUTA_SECUNDARIA',
                  'COBERTURA_BRUTA_MEDIA','DESERCIÓN','DESERCIÓN_TRANSICIÓN','DESERCIÓN_PRIMARIA',
                  'DESERCIÓN_SECUNDARIA','DESERCIÓN_MEDIA','APROBACIÓN','APROBACIÓN_TRANSICIÓN',
                  'APROBACIÓN_PRIMARIA','APROBACIÓN_SECUNDARIA','APROBACIÓN_MEDIA','REPROBACIÓN',
                  'REPROBACIÓN_TRANSICIÓN','REPROBACIÓN_PRIMARIA','REPROBACIÓN_SECUNDARIA','REPROBACIÓN_MEDIA',
                  'REPITENCIA','REPITENCIA_TRANSICIÓN','REPITENCIA_PRIMARIA','REPITENCIA_SECUNDARIA',
                  'REPITENCIA_MEDIA']


In [8]:
MEN_Grupos=MEN.groupby(['GRUPO','AÑO']).apply(
    lambda x: pd.Series(np.average(x[MEN_ESTADISTICAS], weights=x["POBLACIÓN_5_16"], axis=0), MEN_ESTADISTICAS))
MEN_Grupos = MEN_Grupos.reset_index()

In [9]:
MEN=MEN.sort_values(by=['GRUPO','COD_MUN','AÑO'])

with pd.ExcelWriter("MEN_ponderado_.xlsx") as writer:
    
    MEN.to_excel(writer, sheet_name='Municipios')
    MEN_Grupos.to_excel(writer, sheet_name='Grupos')

In [10]:
Matricula10_17 = pd.read_csv(
    "datos educacion media/MEN_MATRICULA_EN_EDUCACION_EN_PREESCOLAR__B_SICA_Y_MEDIA_2010_A_2017.csv",
    low_memory=False)

Matricula18_21 = pd.read_csv(
    "datos educacion media/MEN_MATRICULA_EN_EDUCACION_EN_PREESCOLAR__B_SICA_Y_MEDIA_2018_2021.csv",
    low_memory=False)

Matricula = pd.concat([Matricula10_17,Matricula18_21])
Matricula.rename(columns = {"COD_DANE_MUNICIPIO":"COD_MUN",'ANNO_INF':'AÑO'}, inplace = True)


In [11]:
Matricula = Matricula[['COD_MUN','MUNICIPIO','AÑO',"SECTOR",
                       "TIPO_JORNADA","GRADO",'TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']]

Matricula = Matricula.loc[
    (Matricula['COD_MUN'] == 11001) | (Matricula['COD_MUN'] == 5001)  | (Matricula['COD_MUN'] == 76001) | 
    (Matricula['COD_MUN'] == 8001)  | (Matricula['COD_MUN'] == 13001) | (Matricula['COD_MUN'] == 44001) | 
    (Matricula['COD_MUN'] == 23001) | (Matricula['COD_MUN'] == 47001) | (Matricula['COD_MUN'] == 20001) | 
    (Matricula['COD_MUN'] == 70001) | (Matricula['COD_MUN'] == 68001) | (Matricula['COD_MUN'] == 54001) | 
    (Matricula['COD_MUN'] == 73001) | (Matricula['COD_MUN'] == 50001) | (Matricula['COD_MUN'] == 66001) | 
    (Matricula['COD_MUN'] == 17001) | (Matricula['COD_MUN'] == 52001)]

Matricula = Matricula.loc[
    (Matricula["GRADO"]=='Prejardín') | (Matricula["GRADO"]=="Jardín I") | (Matricula["GRADO"]=="Prejardin II") | 
    (Matricula['GRADO']=="Transición") | (Matricula['GRADO']=="Primero") | (Matricula['GRADO']=="Segundo") | 
    (Matricula['GRADO']=="Tercero") | (Matricula['GRADO']=="Cuarto") | (Matricula['GRADO']=="Quinto") | 
    (Matricula['GRADO']=="Sexto") | (Matricula['GRADO']=="Septimo") | (Matricula['GRADO']=="Octavo") | 
    (Matricula['GRADO']=="Noveno") | (Matricula['GRADO']=="Decimo") | (Matricula['GRADO']=="Once") ]

Matricula = Matricula.loc[(Matricula["TIPO_JORNADA"]!="Fin de Semana") & (Matricula["TIPO_JORNADA"]!="Nocturna")]

Matricula['TIPO_JORNADA'] = np.where(Matricula['TIPO_JORNADA']=="Mañana",'Mañana/Tarde', Matricula['TIPO_JORNADA'])
Matricula['TIPO_JORNADA'] = np.where(Matricula['TIPO_JORNADA']=="Tarde",'Mañana/Tarde', Matricula['TIPO_JORNADA'])


conditions = [
    (Matricula["GRADO"]=='Prejardín') | (Matricula["GRADO"]=="Jardín I") | (Matricula["GRADO"]=="Prejardin II") |
    (Matricula["GRADO"]=="Transición"),
    (Matricula["GRADO"]=='Primero') | (Matricula["GRADO"]=="Segundo") | (Matricula["GRADO"]=="Tercero") | 
    (Matricula["GRADO"]=="Cuarto") |( Matricula["GRADO"]=="Quinto"),
    (Matricula["GRADO"]=='Sexto') | (Matricula["GRADO"]=="Septimo") | (Matricula["GRADO"]=="Octavo") |
    (Matricula["GRADO"]=="Noveno"),
    (Matricula["GRADO"]=='Decimo') | (Matricula["GRADO"]=="Once")
             ]

choices = ['PREESCOLAR', 'BASICA PRIMARIA', 'BASICA SECUNDARIA','MEDIA']

Matricula['NIVEL_EDUCATIVO'] = np.select(conditions, choices)



Promedio ponderado

In [12]:
Matricula_Sector=Matricula.groupby(['COD_MUN',"SECTOR","NIVEL_EDUCATIVO",'AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'nunique', 'CODIGO_DANE_SEDE': 'nunique'})
Matricula_Sector = Matricula_Sector.reset_index()
Matricula_Sector = Matricula_Sector.merge(MEN_Poblacion_5_16, on=['COD_MUN','AÑO'])

conditions = [
    (Matricula_Sector['COD_MUN'] == 11001) | (Matricula_Sector['COD_MUN'] == 5001) | 
    (Matricula_Sector['COD_MUN'] == 76001),
    (Matricula_Sector['COD_MUN'] == 8001),
    (Matricula_Sector['COD_MUN'] == 13001) | (Matricula_Sector['COD_MUN'] == 44001) | 
    (Matricula_Sector['COD_MUN'] == 23001) | (Matricula_Sector['COD_MUN'] == 47001) | 
    (Matricula_Sector['COD_MUN'] == 20001) | (Matricula_Sector['COD_MUN'] == 70001),
    (Matricula_Sector['COD_MUN'] == 68001) | (Matricula_Sector['COD_MUN'] == 54001) | 
    (Matricula_Sector['COD_MUN'] == 73001) | (Matricula_Sector['COD_MUN'] == 50001) | 
    (Matricula_Sector['COD_MUN'] == 66001) | (Matricula_Sector['COD_MUN'] == 17001) | 
    (Matricula_Sector['COD_MUN'] == 52001)
             ]

choices = ['PRINCIPALES', 'BARRANQUILLA', 'CARIBE','OTRAS']
Matricula_Sector["GRUPO"]= np.select(conditions, choices)

Matricula_Sector_grupos=Matricula_Sector.groupby(['SECTOR','NIVEL_EDUCATIVO','GRUPO','AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'sum', 'CODIGO_DANE_SEDE': 'sum'})



In [13]:
#Matricula_Sector_grupos=Matricula_Sector.groupby(['SECTOR','NIVEL_EDUCATIVO','GRUPO','AÑO']).apply(
#    lambda x: pd.Series(np.average(x[['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']], weights=x["POBLACIÓN_5_16"], axis=0),
#                        ['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']))  

In [14]:
Matricula_TipoJornada=Matricula.groupby(['COD_MUN',"TIPO_JORNADA","NIVEL_EDUCATIVO",'AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'nunique', 'CODIGO_DANE_SEDE': 'nunique'})
Matricula_TipoJornada = Matricula_TipoJornada.reset_index()
Matricula_TipoJornada = Matricula_TipoJornada.merge(MEN_Poblacion_5_16, on=['COD_MUN','AÑO'])

conditions = [
    (Matricula_TipoJornada['COD_MUN'] == 11001) | (Matricula_TipoJornada['COD_MUN'] == 5001) | 
    (Matricula_TipoJornada['COD_MUN'] == 76001),
    (Matricula_TipoJornada['COD_MUN'] == 8001),
    (Matricula_TipoJornada['COD_MUN'] == 13001) | (Matricula_TipoJornada['COD_MUN'] == 44001) | 
    (Matricula_TipoJornada['COD_MUN'] == 23001) | (Matricula_TipoJornada['COD_MUN'] == 47001) | 
    (Matricula_TipoJornada['COD_MUN'] == 20001) | (Matricula_TipoJornada['COD_MUN'] == 70001),
    (Matricula_TipoJornada['COD_MUN'] == 68001) | (Matricula_TipoJornada['COD_MUN'] == 54001) | 
    (Matricula_TipoJornada['COD_MUN'] == 73001) | (Matricula_TipoJornada['COD_MUN'] == 50001) | 
    (Matricula_TipoJornada['COD_MUN'] == 66001) | (Matricula_TipoJornada['COD_MUN'] == 17001) | 
    (Matricula_TipoJornada['COD_MUN'] == 52001)
             ]

choices = ['PRINCIPALES', 'BARRANQUILLA', 'CARIBE','OTRAS']
Matricula_TipoJornada["GRUPO"]= np.select(conditions, choices)

Matricula_TipoJornada_grupos=Matricula_TipoJornada.groupby(['TIPO_JORNADA','NIVEL_EDUCATIVO','GRUPO','AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'sum', 'CODIGO_DANE_SEDE': 'sum'})


In [15]:
#Matricula_TipoJornada_grupos=Matricula_TipoJornada.groupby(['TIPO_JORNADA','NIVEL_EDUCATIVO','GRUPO','AÑO']).apply(
#    lambda x: pd.Series(np.average(x[['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']], weights=x["POBLACIÓN_5_16"], axis=0),
#                        ['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']))  

In [16]:
Matricula_Sector=Matricula_Sector.sort_values(by=['SECTOR','NIVEL_EDUCATIVO','GRUPO','COD_MUN','AÑO'])
Matricula_TipoJornada=Matricula_TipoJornada.sort_values(by=['TIPO_JORNADA','NIVEL_EDUCATIVO','GRUPO','COD_MUN','AÑO'])

with pd.ExcelWriter("Matricula MEN.xlsx") as writer:
    
    Matricula_Sector.to_excel(writer, sheet_name='Sector Municipios')
    Matricula_Sector_grupos.to_excel(writer, sheet_name='Sector Grupos')
    Matricula_TipoJornada.to_excel(writer, sheet_name='Jornada Municipios')
    Matricula_TipoJornada_grupos.to_excel(writer, sheet_name='Jornada Grupos')
    

Matricula Basica

In [17]:
Matricula_Basica = Matricula[(Matricula['NIVEL_EDUCATIVO']=='BASICA PRIMARIA') | (Matricula['NIVEL_EDUCATIVO']=='BASICA SECUNDARIA')]
Matricula_Basica_Sector=Matricula_Basica.groupby(['COD_MUN',"SECTOR",'AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'nunique', 'CODIGO_DANE_SEDE': 'nunique'})
Matricula_Basica_Sector = Matricula_Basica_Sector.reset_index()
Matricula_Basica_Sector = Matricula_Basica_Sector.merge(MEN_Poblacion_5_16, on=['COD_MUN','AÑO'])

conditions = [
    (Matricula_Basica_Sector['COD_MUN'] == 11001) | (Matricula_Basica_Sector['COD_MUN'] == 5001) | 
    (Matricula_Basica_Sector['COD_MUN'] == 76001),
    (Matricula_Basica_Sector['COD_MUN'] == 8001),
    (Matricula_Basica_Sector['COD_MUN'] == 13001) | (Matricula_Basica_Sector['COD_MUN'] == 44001) | 
    (Matricula_Basica_Sector['COD_MUN'] == 23001) | (Matricula_Basica_Sector['COD_MUN'] == 47001) | 
    (Matricula_Basica_Sector['COD_MUN'] == 20001) | (Matricula_Basica_Sector['COD_MUN'] == 70001),
    (Matricula_Basica_Sector['COD_MUN'] == 68001) | (Matricula_Basica_Sector['COD_MUN'] == 54001) | 
    (Matricula_Basica_Sector['COD_MUN'] == 73001) | (Matricula_Basica_Sector['COD_MUN'] == 50001) | 
    (Matricula_Basica_Sector['COD_MUN'] == 66001) | (Matricula_Basica_Sector['COD_MUN'] == 17001) | 
    (Matricula_Basica_Sector['COD_MUN'] == 52001)
             ]

choices = ['PRINCIPALES', 'BARRANQUILLA', 'CARIBE','OTRAS']
Matricula_Basica_Sector["GRUPO"]= np.select(conditions, choices)

#Matricula_Basica_Sector_grupos=Matricula_Basica_Sector.groupby(['SECTOR','GRUPO','AÑO']).apply(
#    lambda x: pd.Series(np.average(x[['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']], weights=x["POBLACIÓN_5_16"], axis=0),
#                        ['TOTAL_MATRICULA','CODIGO_DANE', 'CODIGO_DANE_SEDE']))  

Matricula_Basica_Sector_grupos=Matricula_Basica_Sector.groupby(['SECTOR','GRUPO','AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'sum', 'CODIGO_DANE_SEDE': 'sum'})


Matricula_Basica_TipoJornada=Matricula_Basica.groupby(['COD_MUN',"TIPO_JORNADA",'AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'nunique', 'CODIGO_DANE_SEDE': 'nunique'})
Matricula_Basica_TipoJornada = Matricula_Basica_TipoJornada.reset_index()
Matricula_Basica_TipoJornada = Matricula_Basica_TipoJornada.merge(MEN_Poblacion_5_16, on=['COD_MUN','AÑO'])

conditions = [
    (Matricula_Basica_TipoJornada['COD_MUN'] == 11001) | (Matricula_Basica_TipoJornada['COD_MUN'] == 5001) | 
    (Matricula_Basica_TipoJornada['COD_MUN'] == 76001),
    (Matricula_Basica_TipoJornada['COD_MUN'] == 8001),
    (Matricula_Basica_TipoJornada['COD_MUN'] == 13001) | (Matricula_Basica_TipoJornada['COD_MUN'] == 44001) | 
    (Matricula_Basica_TipoJornada['COD_MUN'] == 23001) | (Matricula_Basica_TipoJornada['COD_MUN'] == 47001) | 
    (Matricula_Basica_TipoJornada['COD_MUN'] == 20001) | (Matricula_Basica_TipoJornada['COD_MUN'] == 70001),
    (Matricula_Basica_TipoJornada['COD_MUN'] == 68001) | (Matricula_Basica_TipoJornada['COD_MUN'] == 54001) | 
    (Matricula_Basica_TipoJornada['COD_MUN'] == 73001) | (Matricula_Basica_TipoJornada['COD_MUN'] == 50001) | 
    (Matricula_Basica_TipoJornada['COD_MUN'] == 66001) | (Matricula_Basica_TipoJornada['COD_MUN'] == 17001) | 
    (Matricula_Basica_TipoJornada['COD_MUN'] == 52001)
             ]

choices = ['PRINCIPALES', 'BARRANQUILLA', 'CARIBE','OTRAS']
Matricula_Basica_TipoJornada["GRUPO"]= np.select(conditions, choices)

#Matricula_Basica_TipoJornada_grupos=Matricula_Basica_TipoJornada.groupby(['TIPO_JORNADA','GRUPO','AÑO']).apply(
#    lambda x: pd.Series(np.average(x[['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']], weights=x["POBLACIÓN_5_16"], axis=0),
#                        ['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']))  

Matricula_Basica_TipoJornada_grupos=Matricula_Basica_TipoJornada.groupby(['TIPO_JORNADA','GRUPO','AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'sum', 'CODIGO_DANE_SEDE': 'sum'})


Matricula_Basica_Sector=Matricula_Basica_Sector.sort_values(by=['SECTOR','GRUPO','COD_MUN','AÑO'])
Matricula_Basica_TipoJornada=Matricula_Basica_TipoJornada.sort_values(by=['TIPO_JORNADA','GRUPO','COD_MUN','AÑO'])

with pd.ExcelWriter("Matricula_Jornada_Basica.xlsx") as writer:
    
    Matricula_Basica_Sector.to_excel(writer, sheet_name='Sector Municipios')
    Matricula_Basica_Sector_grupos.to_excel(writer, sheet_name='Sector Grupos')
    Matricula_Basica_TipoJornada.to_excel(writer, sheet_name='Jornada Municipios')
    Matricula_Basica_TipoJornada_grupos.to_excel(writer, sheet_name='Jornada Grupos')
    

Matricula General

In [18]:
Matricula_total=Matricula.groupby(['COD_MUN',"NIVEL_EDUCATIVO",'AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'nunique', 'CODIGO_DANE_SEDE': 'nunique'})
Matricula_total = Matricula_total.reset_index()
Matricula_total = Matricula_total.merge(MEN_Poblacion_5_16, on=['COD_MUN','AÑO'])

conditions = [
    (Matricula_total['COD_MUN'] == 11001) | (Matricula_total['COD_MUN'] == 5001) | 
    (Matricula_total['COD_MUN'] == 76001),
    (Matricula_total['COD_MUN'] == 8001),
    (Matricula_total['COD_MUN'] == 13001) | (Matricula_total['COD_MUN'] == 44001) | 
    (Matricula_total['COD_MUN'] == 23001) | (Matricula_total['COD_MUN'] == 47001) | 
    (Matricula_total['COD_MUN'] == 20001) | (Matricula_total['COD_MUN'] == 70001),
    (Matricula_total['COD_MUN'] == 68001) | (Matricula_total['COD_MUN'] == 54001) | 
    (Matricula_total['COD_MUN'] == 73001) | (Matricula_total['COD_MUN'] == 50001) | 
    (Matricula_total['COD_MUN'] == 66001) | (Matricula_total['COD_MUN'] == 17001) | 
    (Matricula_total['COD_MUN'] == 52001)
             ]

choices = ['PRINCIPALES', 'BARRANQUILLA', 'CARIBE','OTRAS']
Matricula_total["GRUPO"]= np.select(conditions, choices)

#Matricula_total_grupos=Matricula_total.groupby(['NIVEL_EDUCATIVO','GRUPO','AÑO']).apply(
#    lambda x: pd.Series(np.average(x[['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']], weights=x["POBLACIÓN_5_16"], axis=0),
#                        ['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']))  

Matricula_total_grupos=Matricula_total.groupby(['NIVEL_EDUCATIVO','GRUPO','AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'sum', 'CODIGO_DANE_SEDE': 'sum'})


Matricula_Basica_total=Matricula_Basica.groupby(['COD_MUN','AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'nunique', 'CODIGO_DANE_SEDE': 'nunique'})
Matricula_Basica_total = Matricula_Basica_total.reset_index()
Matricula_Basica_total = Matricula_Basica_total.merge(MEN_Poblacion_5_16, on=['COD_MUN','AÑO'])

conditions = [
    (Matricula_Basica_total['COD_MUN'] == 11001) | (Matricula_Basica_total['COD_MUN'] == 5001) | 
    (Matricula_Basica_total['COD_MUN'] == 76001),
    (Matricula_Basica_total['COD_MUN'] == 8001),
    (Matricula_Basica_total['COD_MUN'] == 13001) | (Matricula_Basica_total['COD_MUN'] == 44001) | 
    (Matricula_Basica_total['COD_MUN'] == 23001) | (Matricula_Basica_total['COD_MUN'] == 47001) | 
    (Matricula_Basica_total['COD_MUN'] == 20001) | (Matricula_Basica_total['COD_MUN'] == 70001),
    (Matricula_Basica_total['COD_MUN'] == 68001) | (Matricula_Basica_total['COD_MUN'] == 54001) | 
    (Matricula_Basica_total['COD_MUN'] == 73001) | (Matricula_Basica_total['COD_MUN'] == 50001) | 
    (Matricula_Basica_total['COD_MUN'] == 66001) | (Matricula_Basica_total['COD_MUN'] == 17001) | 
    (Matricula_Basica_total['COD_MUN'] == 52001)
             ]

choices = ['PRINCIPALES', 'BARRANQUILLA', 'CARIBE','OTRAS']
Matricula_Basica_total["GRUPO"]= np.select(conditions, choices)

#Matricula_Basica_total_grupos=Matricula_Basica_total.groupby(['GRUPO','AÑO']).apply(
#    lambda x: pd.Series(np.average(x[['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']], weights=x["POBLACIÓN_5_16"], axis=0),
#                        ['TOTAL_MATRICULA','CODIGO_DANE','CODIGO_DANE_SEDE']))  

Matricula_Basica_total_grupos=Matricula_Basica_total.groupby(['GRUPO','AÑO']).agg(
    {'TOTAL_MATRICULA':'sum', 'CODIGO_DANE': 'sum', 'CODIGO_DANE_SEDE': 'sum'})


Matricula_Basica_total=Matricula_Basica_total.sort_values(by=['GRUPO','COD_MUN','AÑO'])

Matricula_total=Matricula_total.sort_values(by=['NIVEL_EDUCATIVO','GRUPO','COD_MUN','AÑO'])

with pd.ExcelWriter("Matricula_Total.xlsx") as writer:
    
    Matricula_total.to_excel(writer, sheet_name='Total Municipios')
    Matricula_total_grupos.to_excel(writer, sheet_name='Total Grupos')
    Matricula_Basica_total.to_excel(writer, sheet_name='Basica Municipios')
    Matricula_Basica_total_grupos.to_excel(writer, sheet_name='Basica Grupos')
    